In [1]:
pip install langchain langchain-community tavily-python requests transformers torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Travel Planner A2A Multi-Agent System
# Building a travel planner app using Agent-to-Agent protocol

# Install required packages first
# !pip install langchain langchain-community tavily-python requests transformers torch
# With start date only await travel_interface.plan_trip('Rajasthan, India', start_date='2024-11-01')
# With custom dates await travel_interface.plan_trip('Goa, India', '2024-12-15', '2024-12-20')
# With start date only await travel_interface.plan_trip('Rajasthan, India', start_date='2024-11-01')
import os
import json
import requests
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
import asyncio
from dataclasses import dataclass
import random

# LangChain imports
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import BaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

# Hugging Face imports
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Weather and Search APIs
import tavily

print("🚀 Initializing Travel Planner A2A System...")

# =============================================================================
# CONFIGURATION AND SETUP
# =============================================================================

class Config:
    """Configuration for the A2A Travel Planner"""
    
    # API Keys (set these in your environment)
    WEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY", "668c522e2d3b4e5bb4e71955252807")
    TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "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")
    
    # Hugging Face model
    HF_MODEL = "microsoft/DialoGPT-medium"  # Lightweight conversational model

config = Config()

# =============================================================================
# A2A PROTOCOL BASE CLASSES
# =============================================================================

@dataclass
class A2AMessage:
    """Standard message format for Agent-to-Agent communication"""
    sender_id: str
    receiver_id: str
    message_type: str
    content: Dict[str, Any]
    timestamp: str
    request_id: str

class A2AAgent:
    """Base class for A2A compatible agents"""
    
    def __init__(self, agent_id: str, capabilities: List[str]):
        self.agent_id = agent_id
        self.capabilities = capabilities
        self.message_history = []
    
    async def process_message(self, message: A2AMessage) -> A2AMessage:
        """Process incoming A2A message and return response"""
        raise NotImplementedError
    
    def create_response(self, original_msg: A2AMessage, content: Dict[str, Any], 
                       msg_type: str = "response") -> A2AMessage:
        """Create a response message"""
        return A2AMessage(
            sender_id=self.agent_id,
            receiver_id=original_msg.sender_id,
            message_type=msg_type,
            content=content,
            timestamp=datetime.now().isoformat(),
            request_id=original_msg.request_id
        )

# =============================================================================
# WEATHER AGENT
# =============================================================================

class WeatherAgent(A2AAgent):
    """A2A Agent for weather information retrieval"""
    
    def __init__(self):
        super().__init__(
            agent_id="weather_agent",
            capabilities=["current_weather", "weather_forecast", "weather_analysis"]
        )
        self.api_key = config.WEATHER_API_KEY
        self.base_url = "http://api.openweathermap.org/data/2.5"
    
    async def process_message(self, message: A2AMessage) -> A2AMessage:
        """Process weather-related requests"""
        try:
            if message.message_type == "weather_request":
                location = message.content.get("location")
                days = message.content.get("days", 3)
                start_date_str = message.content.get("start_date")
                
                # Parse start date if provided
                start_date = None
                if start_date_str:
                    try:
                        start_date = datetime.fromisoformat(start_date_str)
                    except:
                        start_date = datetime.now()
                
                weather_data = await self.get_weather_forecast(location, days, start_date)
                
                return self.create_response(
                    message,
                    {
                        "weather_data": weather_data,
                        "analysis": self.analyze_weather(weather_data),
                        "status": "success"
                    }
                )
            else:
                return self.create_response(
                    message,
                    {"error": "Unsupported message type", "status": "error"}
                )
                
        except Exception as e:
            return self.create_response(
                message,
                {"error": str(e), "status": "error"}
            )
    
    async def get_weather_forecast(self, location: str, days: int = 3, start_date: datetime = None) -> Dict[str, Any]:
        """Get weather forecast for location with optional start date"""
        try:
            # Use provided start_date or default to today
            if start_date is None:
                start_date = datetime.now()
            
            # For demo purposes, using mock data if API key not available
            if self.api_key == "your_weather_api_key":
                return self.get_mock_weather_data(location, days, start_date)
            
            # Current weather
            current_url = f"{self.base_url}/weather"
            current_params = {
                "q": location,
                "appid": self.api_key,
                "units": "metric"
            }
            
            current_response = requests.get(current_url, params=current_params)
            current_data = current_response.json()
            
            # Forecast
            forecast_url = f"{self.base_url}/forecast"
            forecast_params = {
                "q": location,
                "appid": self.api_key,
                "units": "metric",
                "cnt": days * 8  # 8 forecasts per day (3-hour intervals)
            }
            
            forecast_response = requests.get(forecast_url, params=forecast_params)
            forecast_data = forecast_response.json()
            
            return {
                "current": current_data,
                "forecast": forecast_data,
                "location": location,
                "trip_start_date": start_date.isoformat(),
                "forecast_days": days
            }
            
        except Exception as e:
            print(f"Weather API error: {e}")
            return self.get_mock_weather_data(location, days, start_date)
    
    def get_mock_weather_data(self, location: str, days: int = 3, start_date: datetime = None) -> Dict[str, Any]:
        """Generate mock weather data for demo with custom dates"""
        if start_date is None:
            start_date = datetime.now()
        
        import random
        
        conditions = ["clear", "partly_cloudy", "cloudy", "rain", "thunderstorm"]
        temps = [25, 28, 22, 30, 18]
        
        condition = random.choice(conditions)
        temp = random.choice(temps)
        
        # Generate forecast for specified days starting from start_date
        forecast_list = []
        for i in range(days):
            forecast_date = start_date + timedelta(days=i)
            forecast_list.append({
                "dt_txt": forecast_date.strftime("%Y-%m-%d 12:00:00"),
                "weather": [{"main": random.choice(conditions).replace("_", " ").title()}],
                "main": {"temp": random.randint(20, 32)},
                "date": forecast_date.strftime("%Y-%m-%d")
            })
        
        return {
            "current": {
                "weather": [{"main": condition.replace("_", " ").title(), "description": f"{condition} sky"}],
                "main": {"temp": temp, "humidity": random.randint(40, 80)},
                "name": location
            },
            "forecast": {
                "list": forecast_list
            },
            "location": location,
            "trip_start_date": start_date.isoformat(),
            "forecast_days": days
        }
    
    def analyze_weather(self, weather_data: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze weather conditions for travel recommendations"""
        current = weather_data.get("current", {})
        forecast = weather_data.get("forecast", {})
        
        current_condition = current.get("weather", [{}])[0].get("main", "").lower()
        current_temp = current.get("main", {}).get("temp", 25)
        
        # Determine activity suitability
        outdoor_friendly = current_condition not in ["rain", "thunderstorm", "snow"]
        temp_comfortable = 18 <= current_temp <= 35
        
        activity_type = "outdoor" if outdoor_friendly and temp_comfortable else "indoor"
        
        # Get trip dates from weather data
        trip_start = weather_data.get("trip_start_date", datetime.now().isoformat())
        forecast_days = weather_data.get("forecast_days", 3)
        
        return {
            "current_condition": current_condition,
            "temperature": current_temp,
            "outdoor_friendly": outdoor_friendly,
            "recommended_activity_type": activity_type,
            "weather_summary": f"Current weather in {weather_data.get('location', 'the area')} is {current_condition} with {current_temp}°C",
            "trip_start_date": trip_start,
            "trip_duration_days": forecast_days
        }

# =============================================================================
# SEARCH AGENT (Using Tavily)
# =============================================================================

class SearchAgent(A2AAgent):
    """A2A Agent for web search using Tavily"""
    
    def __init__(self):
        super().__init__(
            agent_id="search_agent",
            capabilities=["web_search", "travel_search", "activity_search"]
        )
        self.tavily_client = None
        if config.TAVILY_API_KEY != "your_tavily_api_key":
            self.tavily_client = tavily.TavilyClient(api_key=config.TAVILY_API_KEY)
    
    async def process_message(self, message: A2AMessage) -> A2AMessage:
        """Process search requests"""
        try:
            if message.message_type == "search_request":
                query = message.content.get("query")
                location = message.content.get("location")
                activity_type = message.content.get("activity_type", "outdoor")
                
                search_results = await self.search_activities(query, location, activity_type)
                
                return self.create_response(
                    message,
                    {
                        "search_results": search_results,
                        "processed_recommendations": self.process_search_results(search_results),
                        "status": "success"
                    }
                )
            else:
                return self.create_response(
                    message,
                    {"error": "Unsupported message type", "status": "error"}
                )
                
        except Exception as e:
            return self.create_response(
                message,
                {"error": str(e), "status": "error"}
            )
    
    async def search_activities(self, query: str, location: str, activity_type: str) -> List[Dict[str, Any]]:
        """Search for activities based on location and type"""
        try:
            if self.tavily_client:
                search_query = f"{activity_type} activities {location} {query} things to do"
                response = self.tavily_client.search(
                    query=search_query,
                    search_depth="basic",
                    max_results=5
                )
                return response.get("results", [])
            else:
                # Mock search results for demo
                return self.get_mock_search_results(location, activity_type)
                
        except Exception as e:
            print(f"Search error: {e}")
            return self.get_mock_search_results(location, activity_type)
    
    def get_mock_search_results(self, location: str, activity_type: str) -> List[Dict[str, Any]]:
        """Generate mock search results for demo"""
        if activity_type == "outdoor":
            activities = [
                f"Visit {location} beaches and coastal areas",
                f"Explore {location} national parks and hiking trails",
                f"Take a boat tour around {location}",
                f"Visit {location} outdoor markets and street food",
                f"Go on a {location} photography tour"
            ]
        else:
            activities = [
                f"Visit {location} museums and cultural centers",
                f"Explore {location} shopping malls and indoor markets",
                f"Try {location} cooking classes and food tours",
                f"Visit {location} art galleries and exhibitions",
                f"Experience {location} spa and wellness centers"
            ]
        
        return [
            {
                "title": activity,
                "content": f"Great {activity_type} activity in {location}. {activity} offers amazing experiences.",
                "url": f"https://example.com/{activity.replace(' ', '-').lower()}",
                "score": 0.9
            } for activity in activities
        ]
    
    def process_search_results(self, results: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Process and rank search results"""
        processed = []
        for result in results[:5]:  # Top 5 results
            processed.append({
                "title": result.get("title", ""),
                "description": result.get("content", "")[:200] + "...",
                "url": result.get("url", ""),
                "relevance_score": result.get("score", 0.5)
            })
        return sorted(processed, key=lambda x: x["relevance_score"], reverse=True)

# =============================================================================
# HUGGING FACE LLM AGENT
# =============================================================================

class HuggingFaceLLMAgent(A2AAgent):
    """A2A Agent using Hugging Face models for text generation"""
    
    def __init__(self):
        super().__init__(
            agent_id="hf_llm_agent",
            capabilities=["text_generation", "summarization", "travel_planning"]
        )
        self.setup_model()
    
    def setup_model(self):
        """Initialize Hugging Face model"""
        try:
            # Use a lightweight model for demo
            self.tokenizer = AutoTokenizer.from_pretrained(config.HF_MODEL)
            self.model = AutoModelForCausalLM.from_pretrained(config.HF_MODEL)
            
            # Add padding token if not present
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            self.generator = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                device=0 if torch.cuda.is_available() else -1,
                max_length=512,
                do_sample=True,
                temperature=0.7
            )
            print("✅ Hugging Face model loaded successfully")
        except Exception as e:
            print(f"❌ Error loading Hugging Face model: {e}")
            self.generator = None
    
    async def process_message(self, message: A2AMessage) -> A2AMessage:
        """Process text generation requests"""
        try:
            if message.message_type == "generate_itinerary":
                weather_data = message.content.get("weather_data")
                search_results = message.content.get("search_results")
                location = message.content.get("location")
                
                itinerary = await self.generate_travel_itinerary(weather_data, search_results, location)
                
                return self.create_response(
                    message,
                    {
                        "itinerary": itinerary,
                        "status": "success"
                    }
                )
            else:
                return self.create_response(
                    message,
                    {"error": "Unsupported message type", "status": "error"}
                )
                
        except Exception as e:
            return self.create_response(
                message,
                {"error": str(e), "status": "error"}
            )
    
    async def generate_travel_itinerary(self, weather_data: Dict, search_results: List[Dict], location: str, trip_dates: Dict = None) -> Dict[str, Any]:
        """Generate comprehensive travel itinerary with custom dates"""
        try:
            # Extract key information
            weather_analysis = weather_data.get("analysis", {})
            activity_type = weather_analysis.get("recommended_activity_type", "outdoor")
            weather_summary = weather_analysis.get("weather_summary", "")
            
            # Get trip dates
            if trip_dates:
                start_date = datetime.fromisoformat(trip_dates["start_date"])
                end_date = datetime.fromisoformat(trip_dates["end_date"])
                duration = (end_date - start_date).days + 1
            else:
                start_date = datetime.fromisoformat(weather_analysis.get("trip_start_date", datetime.now().isoformat()))
                duration = weather_analysis.get("trip_duration_days", 3)
                end_date = start_date + timedelta(days=duration-1)
            
            activities = [result.get("title", "") for result in search_results[:3]]
            
            # Create prompt for itinerary generation
            prompt = f"""Create a travel itinerary for {location}.
Trip dates: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} ({duration} days)
Weather: {weather_summary}
Recommended activity type: {activity_type}
Top activities: {', '.join(activities)}

Generate a {duration}-day detailed itinerary:"""

            if self.generator:
                # Generate using Hugging Face model
                generated = self.generator(
                    prompt,
                    max_length=300,
                    num_return_sequences=1,
                    pad_token_id=self.tokenizer.eos_token_id
                )
                generated_text = generated[0]["generated_text"].replace(prompt, "").strip()
            else:
                # Fallback to template-based generation
                generated_text = self.generate_template_itinerary(location, activity_type, activities, weather_summary, start_date, duration)
            
            return {
                "location": location,
                "trip_start_date": start_date.isoformat(),
                "trip_end_date": end_date.isoformat(),
                "trip_duration": duration,
                "weather_info": weather_summary,
                "activity_type": activity_type,
                "generated_itinerary": generated_text,
                "recommended_activities": activities,
                "generation_method": "hugging_face" if self.generator else "template"
            }
            
        except Exception as e:
            print(f"Itinerary generation error: {e}")
            return self.generate_template_itinerary(location, activity_type, activities, weather_summary, datetime.now(), 3)
    
    def generate_template_itinerary(self, location: str, activity_type: str, activities: List[str], weather_summary: str, start_date: datetime, duration: int) -> Dict[str, Any]:
        """Generate itinerary using template with custom dates and duration"""
        
        # Generate day-by-day itinerary
        itinerary_days = []
        for day in range(duration):
            current_date = start_date + timedelta(days=day)
            day_number = day + 1
            
            if day == 0:  # First day
                day_plan = f"""📅 Day {day_number} ({current_date.strftime('%B %d, %Y')}): Arrival & Local Exploration
• Morning: Arrive in {location} and check into accommodation
• Afternoon: {activities[0] if activities else f'Explore {location} city center'}
• Evening: Try local cuisine at traditional restaurants"""
            
            elif day == duration - 1:  # Last day
                day_plan = f"""📅 Day {day_number} ({current_date.strftime('%B %d, %Y')}): Final Exploration & Departure
• Morning: {activities[min(2, len(activities)-1)] if activities else f'Last-minute shopping in {location}'}
• Afternoon: Pack and prepare for departure
• Evening: Farewell dinner and departure"""
            
            else:  # Middle days
                activity_index = min(day, len(activities)-1) if activities else 0
                day_plan = f"""📅 Day {day_number} ({current_date.strftime('%B %d, %Y')}): Main Attractions
• Morning: {activities[activity_index] if activities else f'Visit {location} main attractions'}
• Afternoon: {f'{activity_type.title()} activities' if activity_type else 'Sightseeing tour'}
• Evening: Local markets and cultural experiences"""
            
            itinerary_days.append(day_plan)
        
        full_itinerary = f"""
🌟 {duration}-Day {location} Travel Itinerary
📍 Trip Dates: {start_date.strftime('%B %d, %Y')} - {(start_date + timedelta(days=duration-1)).strftime('%B %d, %Y')}

{chr(10).join(itinerary_days)}

🌤️ Weather Consideration: {weather_summary}
💡 Activity Focus: {activity_type.title()} activities recommended

📝 Important Notes:
• Pack appropriate clothing for {activity_type} activities
• Check local weather updates daily
• Book popular attractions in advance
• Try local transportation options
• Keep emergency contacts handy

📱 Daily Reminders:
• Check weather forecast each morning
• Carry appropriate gear for {activity_type} activities
• Stay hydrated and take breaks
• Respect local customs and traditions
        """
        
        return {
            "location": location,
            "trip_start_date": start_date.isoformat(),
            "trip_end_date": (start_date + timedelta(days=duration-1)).isoformat(),
            "trip_duration": duration,
            "weather_info": weather_summary,
            "activity_type": activity_type,
            "generated_itinerary": full_itinerary.strip(),
            "recommended_activities": activities,
            "generation_method": "template"
        }

# =============================================================================
# MAIN TRAVEL PLANNER AGENT (ORCHESTRATOR)
# =============================================================================

class TravelPlannerAgent(A2AAgent):
    """Main orchestrator agent for the travel planner system"""
    
    def __init__(self):
        super().__init__(
            agent_id="travel_planner",
            capabilities=["travel_orchestration", "agent_coordination", "itinerary_compilation"]
        )
        
        # Initialize sub-agents
        self.weather_agent = WeatherAgent()
        self.search_agent = SearchAgent()
        self.llm_agent = HuggingFaceLLMAgent()
        
        print("✅ All A2A agents initialized successfully")
    
    async def create_travel_plan(self, destination: str, start_date: str = None, end_date: str = None, preferences: Dict[str, Any] = None) -> Dict[str, Any]:
        """Main method to create comprehensive travel plan with custom dates"""
        print(f"\n🎯 Creating travel plan for: {destination}")
        
        # Handle date inputs
        try:
            if start_date:
                trip_start = datetime.fromisoformat(start_date) if isinstance(start_date, str) else start_date
            else:
                trip_start = datetime.now()
            
            if end_date:
                trip_end = datetime.fromisoformat(end_date) if isinstance(end_date, str) else end_date
                days = (trip_end - trip_start).days + 1
            else:
                days = 3  # Default duration
                trip_end = trip_start + timedelta(days=days-1)
                
            print(f"📅 Trip dates: {trip_start.strftime('%Y-%m-%d')} to {trip_end.strftime('%Y-%m-%d')} ({days} days)")
            
        except Exception as e:
            print(f"⚠️ Date parsing error: {e}. Using default dates.")
            trip_start = datetime.now()
            days = 3
            trip_end = trip_start + timedelta(days=days-1)
        
        request_id = f"travel_plan_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        try:
            # Step 1: Get weather information
            print("📡 Step 1: Fetching weather information...")
            weather_request = A2AMessage(
                sender_id=self.agent_id,
                receiver_id="weather_agent",
                message_type="weather_request",
                content={"location": destination, "days": days, "start_date": trip_start.isoformat()},
                timestamp=datetime.now().isoformat(),
                request_id=request_id
            )
            
            weather_response = await self.weather_agent.process_message(weather_request)
            weather_data = weather_response.content
            
            if weather_data.get("status") != "success":
                raise Exception(f"Weather request failed: {weather_data.get('error')}")
            
            print(f"✅ Weather data retrieved: {weather_data['analysis']['weather_summary']}")
            
            # Step 2: Search for activities based on weather
            print("🔍 Step 2: Searching for activities...")
            activity_type = weather_data["analysis"]["recommended_activity_type"]
            
            search_request = A2AMessage(
                sender_id=self.agent_id,
                receiver_id="search_agent",
                message_type="search_request",
                content={
                    "query": "travel attractions tourism",
                    "location": destination,
                    "activity_type": activity_type
                },
                timestamp=datetime.now().isoformat(),
                request_id=request_id
            )
            
            search_response = await self.search_agent.process_message(search_request)
            search_data = search_response.content
            
            if search_data.get("status") != "success":
                raise Exception(f"Search request failed: {search_data.get('error')}")
            
            print(f"✅ Found {len(search_data['processed_recommendations'])} activity recommendations")
            
            # Step 3: Generate comprehensive itinerary
            print("📝 Step 3: Generating comprehensive itinerary...")
            itinerary_request = A2AMessage(
                sender_id=self.agent_id,
                receiver_id="hf_llm_agent",
                message_type="generate_itinerary",
                content={
                    "weather_data": weather_data,
                    "search_results": search_data["processed_recommendations"],
                    "location": destination,
                    "trip_dates": {
                        "start_date": trip_start.isoformat(),
                        "end_date": trip_end.isoformat()
                    }
                },
                timestamp=datetime.now().isoformat(),
                request_id=request_id
            )
            
            itinerary_response = await self.llm_agent.process_message(itinerary_request)
            itinerary_data = itinerary_response.content
            
            if itinerary_data.get("status") != "success":
                raise Exception(f"Itinerary generation failed: {itinerary_data.get('error')}")
            
            print("✅ Comprehensive itinerary generated")
            
            # Compile final result
            final_plan = {
                "destination": destination,
                "trip_start_date": trip_start.strftime('%Y-%m-%d'),
                "trip_end_date": trip_end.strftime('%Y-%m-%d'),
                "trip_duration_days": days,
                "request_id": request_id,
                "timestamp": datetime.now().isoformat(),
                "weather_analysis": weather_data["analysis"],
                "activity_recommendations": search_data["processed_recommendations"],
                "comprehensive_itinerary": itinerary_data["itinerary"],
                "agent_collaboration": {
                    "weather_agent": "success",
                    "search_agent": "success",
                    "huggingface_llm_agent": "success"
                },
                "privacy_note": "All processing done locally/privately without sharing personal data with third parties"
            }
            
            print("🎉 Travel plan completed successfully!")
            return final_plan
            
        except Exception as e:
            print(f"❌ Error creating travel plan: {e}")
            return {
                "destination": destination,
                "trip_start_date": trip_start.strftime('%Y-%m-%d') if 'trip_start' in locals() else None,
                "trip_end_date": trip_end.strftime('%Y-%m-%d') if 'trip_end' in locals() else None,
                "error": str(e),
                "status": "failed",
                "timestamp": datetime.now().isoformat()
            }

# =============================================================================
# DEMO AND TESTING
# =============================================================================

async def demo_travel_planner():
    """Demonstrate the A2A Travel Planner system"""
    print("🚀 Starting A2A Travel Planner Demo...")
    print("=" * 60)
    
    # Initialize the main travel planner
    planner = TravelPlannerAgent()
    
    # Test destinations
    destinations = ["Kerala, India", "Goa, India", "Rajasthan, India"]
    
    for destination in destinations:
        print(f"\n{'='*60}")
        print(f"🎯 PLANNING TRIP TO: {destination}")
        print(f"{'='*60}")
        
        # Create travel plan
        plan = await planner.create_travel_plan(destination)
        
        if plan.get("status") != "failed":
            # Display results
            print(f"\n📍 Destination: {plan['destination']}")
            print(f"📅 Trip Duration: {plan.get('trip_duration_days', 'N/A')} days")
            print(f"🌤️ Weather: {plan['weather_analysis']['weather_summary']}")
            print(f"🎯 Activity Type: {plan['weather_analysis']['recommended_activity_type']}")
            
            print(f"\n🏆 Top Recommendations:")
            for i, activity in enumerate(plan['activity_recommendations'][:3], 1):
                print(f"  {i}. {activity['title']}")
            
            print(f"\n📋 Generated Itinerary:")
            print(plan['comprehensive_itinerary']['generated_itinerary'])
            
            print(f"\n🤝 Agent Collaboration Status:")
            for agent, status in plan['agent_collaboration'].items():
                print(f"  • {agent}: {status}")
            
            print(f"\n🔒 {plan['privacy_note']}")
        else:
            print(f"❌ Failed to create plan: {plan.get('error')}")
        
        print("\n" + "="*60)
        await asyncio.sleep(1)  # Small delay between requests

# =============================================================================
# JUPYTER NOTEBOOK INTERFACE
# =============================================================================

class TravelPlannerInterface:
    """Interactive interface for Jupyter notebook"""
    
    def __init__(self):
        self.planner = TravelPlannerAgent()
        print("🎯 Travel Planner A2A System Initialized!")
        print("📱 Ready to create amazing travel plans!")
    
    async def plan_trip(self, destination: str, start_date: str = None, end_date: str = None, show_details: bool = True):
        """Interactive trip planning method with custom dates"""
        if start_date or end_date:
            date_info = f" from {start_date or 'today'} to {end_date or 'auto'}"
            print(f"🌟 Planning your trip to {destination}{date_info}...")
        else:
            print(f"🌟 Planning your trip to {destination}...")
        
        plan = await self.planner.create_travel_plan(destination, start_date, end_date)
        
        if plan.get("status") != "failed":
            if show_details:
                self.display_plan(plan)
            return plan
        else:
            print(f"❌ Sorry, couldn't create plan: {plan.get('error')}")
            return None
    
    def display_plan(self, plan: Dict[str, Any]):
        """Display formatted travel plan"""
        print("\n" + "🎉" * 20)
        print(f"🏖️  TRAVEL PLAN FOR {plan['destination'].upper()}  🏖️")
        print("🎉" * 20)
        
        # Weather info
        weather = plan['weather_analysis']
        print(f"\n🌤️  WEATHER FORECAST")
        print(f"Current: {weather['weather_summary']}")
        print(f"Activity Recommendation: {weather['recommended_activity_type'].title()} activities")
        
        # Top activities
        print(f"\n🎯  TOP RECOMMENDATIONS")
        for i, activity in enumerate(plan['activity_recommendations'][:5], 1):
            print(f"{i:2d}. {activity['title']}")
            print(f"    📊 Relevance: {activity['relevance_score']:.1f}/1.0")
        
        # Itinerary
        print(f"\n📅  YOUR PERSONALIZED ITINERARY")
        print(plan['comprehensive_itinerary']['generated_itinerary'])
        
        # Agent status
        print(f"\n🤖  SYSTEM STATUS")
        for agent, status in plan['agent_collaboration'].items():
            emoji = "✅" if status == "success" else "⚠️" if status == "not_available" else "❌"
            print(f"{emoji} {agent.replace('_', ' ').title()}: {status}")
        
        print(f"\n🔐  Privacy: {plan['privacy_note']}")
        print("\n" + "🎉" * 20)

# Initialize the interface
travel_interface = TravelPlannerInterface()

print("\n" + "="*60)
print("🎯 A2A TRAVEL PLANNER SYSTEM READY!")
print("="*60)
print("\n📚 Usage Examples:")
print("1. await travel_interface.plan_trip('Kerala, India')")
print("2. await travel_interface.plan_trip('Goa, India')")
print("3. await demo_travel_planner()  # Run full demo")
print("\n💡 Features:")
print("• Multi-agent A2A protocol communication")
print("• Weather-based activity recommendations")
print("• Intelligent web search with Tavily")
print("• Hugging Face LLM integration")
print("• Privacy-focused local processing")
print("\n🚀 Ready to explore the world!")

🚀 Initializing Travel Planner A2A System...


Device set to use cuda:0


✅ Hugging Face model loaded successfully
✅ All A2A agents initialized successfully
🎯 Travel Planner A2A System Initialized!
📱 Ready to create amazing travel plans!

🎯 A2A TRAVEL PLANNER SYSTEM READY!

📚 Usage Examples:
1. await travel_interface.plan_trip('Kerala, India')
2. await travel_interface.plan_trip('Goa, India')
3. await demo_travel_planner()  # Run full demo

💡 Features:
• Multi-agent A2A protocol communication
• Weather-based activity recommendations
• Intelligent web search with Tavily
• Hugging Face LLM integration
• Optional Ollama enhancement
• Privacy-focused local processing

🚀 Ready to explore the world!


In [2]:
await travel_interface.plan_trip('Rajasthan, India', start_date='2025-11-01')

🌟 Planning your trip to Rajasthan, India from 2025-11-01 to auto...

🎯 Creating travel plan for: Rajasthan, India
📅 Trip dates: 2025-11-01 to 2025-11-03 (3 days)
📡 Step 1: Fetching weather information...
✅ Weather data retrieved: Current weather in Rajasthan, India is  with 25°C
🔍 Step 2: Searching for activities...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


✅ Found 5 activity recommendations
📝 Step 3: Generating comprehensive itinerary...
✅ Comprehensive itinerary generated
🎉 Travel plan completed successfully!

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
🏖️  TRAVEL PLAN FOR RAJASTHAN, INDIA  🏖️
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

🌤️  WEATHER FORECAST
Current: Current weather in Rajasthan, India is  with 25°C
Activity Recommendation: Outdoor activities

🎯  TOP RECOMMENDATIONS
 1. THE TOP 10 Rajasthan Outdoor Activities (UPDATED 2025) - Viator
    📊 Relevance: 0.8/1.0
 2. THE 10 BEST Outdoor Activities in Rajasthan (Updated 2025)
    📊 Relevance: 0.8/1.0
 3. Adventure Activities & Sports in Rajasthan
    📊 Relevance: 0.7/1.0
 4. Things to do in Rajasthan | My Travel Encounters
    📊 Relevance: 0.7/1.0
 5. 9 Adventure Sports In Rajasthan For Thrill & Excitement In 2025
    📊 Relevance: 0.6/1.0

📅  YOUR PERSONALIZED ITINERARY
2026

🤖  SYSTEM STATUS
✅ Weather Agent: success
✅ Search Agent: success
✅ Huggingface Llm Agent: success

🔐  Privacy: All processing done locally/privatel

{'destination': 'Rajasthan, India',
 'trip_start_date': '2025-11-01',
 'trip_end_date': '2025-11-03',
 'trip_duration_days': 3,
 'request_id': 'travel_plan_20250729_085952',
 'timestamp': '2025-07-29T08:59:57.138823',
 'weather_analysis': {'current_condition': '',
  'temperature': 25,
  'outdoor_friendly': True,
  'recommended_activity_type': 'outdoor',
  'weather_summary': 'Current weather in Rajasthan, India is  with 25°C',
  'trip_start_date': '2025-11-01T00:00:00',
  'trip_duration_days': 3},
 'activity_recommendations': [{'title': 'THE TOP 10 Rajasthan Outdoor Activities (UPDATED 2025) - Viator',
   'description': "You'll visit the Sachhai Mata Temple in Osian, take a camel ride, take a Jeep safari through sand dunes, and dine on an included meal of Rajasthani fare, with...",
   'url': 'https://www.viator.com/Rajasthan-tours/Outdoor-Activities/d4982-g9',
   'relevance_score': 0.83093774},
  {'title': 'THE 10 BEST Outdoor Activities in Rajasthan (Updated 2025)',
   'description': '